# Tutorial for working with Projects

In order to use this notebook you need to have the [rapidminer](https://github.com/rapidminer/python-rapidminer) package installed in your current Python environment. For installation instructions, consult the [README](https://github.com/rapidminer/python-rapidminer).


### Reading and Writing Data from Projects

You can easily read from and write to Altair AI Hub projects. For this you have to clone the project using git. The examples below use the git client via the terminal, so you need these commands to work on your system.

Let's assume, AI Hub is running at localhost and uses the default port. The next two lines will clone the sample-dev project into a subdirectory under the _/work_ folder:

In [ ]:
%cd ~/work
!git clone http://localhost:8080/executions/git/sample-dev.git || (cd sample-dev && git pull origin master && git checkout master && cd ..)

Now you can refer to the project and read the Iris dataset into a pandas DataFrame:

In [ ]:
%cd ~/work
import rapidminer
sample = rapidminer.Project("sample-dev")
iris = sample.read("data/Iris")
iris

Let's create a similar dataset, which we will upload.

In [ ]:
import pandas
from sklearn.datasets import load_wine

sklearn_wine = load_wine()
wine = pandas.DataFrame(sklearn_wine["data"], columns=sklearn_wine["feature_names"])
wine["target"] = sklearn_wine["target"]

Writing a pandas DataFrame is this simple:

In [ ]:
sample.write(wine, "data/wine")

*Remember to commit and push your changes to the Altair AI Hub project.* Uncomment next code block if you want to do so (or use the git panel, shown on the left by default). Note that the git client must have Git LFS support. The LFS url of the repository is set automatically.

In [ ]:
#%cd sample-dev
#!git add data/wine.rmhdf5table
#!git commit -m "creating wine dataset"
#!git push -u origin master
#%cd ..

### Using Connections from a Project

You can access connections of a project easily. You have access to the connection fields and can use them to initiate connection to external systems without the need to input any value on the Python side.

In [ ]:
import rapidminer
%cd ~/work
connections = rapidminer.Connections("sample-dev", server=rapidminer.Server("https://myserver.mycompany.com:8080", authentication_server="https:///myserver.mycompany.com:8081/auth", offline_token="qwert12345", client_secret="qwert12345"))

Here, we already pointed to a [Server](server_examples.ipynb) instance. That is only necessary if you have encrypted connection fields or use the AI Hub Vault to store certain sensitive values.

Let's connect to a PostgreSQL database using one of the connections of the project and read a DataFrame directly from it:

In [ ]:
from sqlalchemy import create_engine
postgres_str = ('postgresql://{username}:{password}@{host}:{port}/{dbname}'
    .format(username=conn.user,
        password=conn.password,
        host=conn.values["host"],
        port=conn.values["port"],
        dbname=db_name
    )
)
cnx = create_engine(postgres_str).raw_connection()
pandas.read_sql_query("SELECT * FROM test_date_types_1", con=cnx)